# Finetuned Random model on Novel Query Dataset

Paper: Single-Cell RNA Sequencing Reveals HIF1A as a Severity-Sensitive Immunological Scar in Circulating Monocytes of Convalescent Comorbidity-Free COVID-19 Patients

Link: https://www.mdpi.com/2073-4409/13/4/300



In [1]:
# %%
import copy
import gc
import json
import os
from pathlib import Path
import shutil
import sys
import time
import traceback
from typing import List, Tuple, Dict, Union, Optional
import warnings
import pandas as pd
# from . import asyn
import pickle
import torch
import anndata
import scanpy as sc
# import scvi
import seaborn as sns
import numpy as np
import wandb
from scipy.sparse import issparse
import matplotlib.pyplot as plt
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from torchtext.vocab import Vocab
from torchtext._torchtext import (
    Vocab as VocabPybind,
)
from sklearn.metrics import confusion_matrix

sys.path.insert(0, "../")
import scgpt as scg
from scgpt.model import TransformerModel, AdversarialDiscriminator
from scgpt.tokenizer import tokenize_and_pad_batch, random_mask_value
from scgpt.loss import (
    masked_mse_loss,
    masked_relative_error,
    criterion_neg_log_bernoulli,
)
from scgpt.tokenizer.gene_tokenizer import GeneVocab
from scgpt.preprocess import Preprocessor
from scgpt import SubsetsBatchSampler
from scgpt.utils import set_seed, category_str2int, eval_scib_metrics
sc.set_figure_params(figsize=(6, 6))
os.environ["KMP_WARNINGS"] = "off"
warnings.filterwarnings('ignore')

/tscc/nfs/home/s5srinivasan/py39env/lib64/python3.9/site-packages/scgpt/model/model.py:21: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")
/tscc/nfs/home/s5srinivasan/py39env/lib64/python3.9/site-packages/scgpt/model/multiomic_model.py:19: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")
/tscc/nfs/home/s5srinivasan/py39env/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))
/tscc/nfs/home/s5srinivasan/py39env/lib64/python3.9/site-packages/scanpy/_settings.py:488: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_formats(*ipython_format)


In [18]:
dataDir = Path(f"/tscc/nfs/home/s5srinivasan/scratch")
projectDir = Path(f"/tscc/nfs/home/s5srinivasan/scgpt-project/covid-annotation-scgpt")

externalQuery = sc.read(dataDir / "may24_covid19.h5ad") # May et al. query dataset

modelPath = Path(f"/tscc/nfs/home/s5srinivasan/scgpt-project/covid-annotation-scgpt/save/randomModel") # use of random model

In [16]:
externalQuery.var

,gene_symbol,gene_id
gene_id,,
ENSG00000085563,ABCB1,ENSG00000085563
ENSG00000103222.0.PolyA_1,ABCC1.0.PolyA_1,ENSG00000103222.0.PolyA_1
ENSG00000103222.0.ref,ABCC1.0.ref,ENSG00000103222.0.ref
ENSG00000103222.1,ABCC1.1,ENSG00000103222.1
ENSG00000106546,AHR,ENSG00000106546
...,...,...
ENSG00000109906,ZBTB16,ENSG00000109906
ENSG00000160685,ZBTB7B,ENSG00000160685
ENSG00000169554.0.PolyA_1,ZEB2.0.PolyA_1,ENSG00000169554.0.PolyA_1


In [15]:
finetunedModel = torch.load(modelPath) # PyTorch load model
finetunedModel.eval()

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

# Reference Mapping Setup

In [23]:
embed_adata = scg.tasks.cell_emb.embed_data(
    externalQuery,
    modelPath,
    gene_col="gene_symbol",
    batch_size=64,
)

scGPT - INFO - match 316/365 genes in vocabulary of size 60697.


Embedding cells: 100%|██████████| 1227/1227 [00:54<00:00, 22.62it/s]
